In [ ]:
import jsonlines
import json
import polars as pl

In [ ]:
# lines contains all log lines
lines = []
with jsonlines.open('logs.jsonl') as reader:
    for log in reader:
        lines.append(json.JSONDecoder().decode(log['line']))
lines[:3]

In [ ]:
# routines contains all lines relevant to routines
routines = list(filter(lambda x: x.get('routine', None) is not None, lines))

routines[:3]

In [ ]:
len(routines)

In [ ]:
# put all routine times into a dictionary
from collections import defaultdict

routine_times = defaultdict(list)

for routine in routines:
    routine_name = routine['routine']
    elapsed_time = routine['elapsedTime']
    routine_times[routine_name].append(elapsed_time)

routine_times = dict(routine_times)

sorted(list(routine_times.keys()))

In [ ]:
# calculate average time for each routine
for routine_key in routine_times:
    routine_times[routine_key] = sum(routine_times[routine_key]) / len(routine_times[routine_key])

routine_times = dict(sorted(routine_times.items(), key=lambda item: item[1], reverse=True))
routine_times